In [ ]:
# === app.py ===
# אפליקציית CLI מלאה: Google Login (OAuth Desktop) → Gemini → Firestore
# פקודות: list / retrieve <index> / exit

from datetime import datetime, timezone
import os, re, requests, sys
from typing import Tuple

# Firestore
from google.cloud import firestore
from google.oauth2 import service_account
from google.cloud import firestore as _fs

# Google OAuth (Desktop app)
from google_auth_oauthlib.flow import InstalledAppFlow

# Gemini
import google.generativeai as genai


# ---------- הגדרות ----------
PROJECT_ID = "global-doodad-471111-s3"   # ← עדכן אם צריך
CREDS_PATH = r"C:\Users\guymk\global-doodad-471111-s3-2cfd774c3ae5.json"  # ← Service Account JSON
CLIENT_SECRETS_FILE = "client_secret_desktop.json"  # ← OAuth Desktop JSON (ליד הקובץ או עדכן לנתיב מלא)

# ---------- Init: Firestore ----------
if not os.path.exists(CREDS_PATH):
    print(f"❌ לא נמצא קובץ Service Account בנתיב: {CREDS_PATH}")
    sys.exit(1)

sa_creds = service_account.Credentials.from_service_account_file(CREDS_PATH)
db = firestore.Client(project=PROJECT_ID, credentials=sa_creds)

# ---------- Init: Gemini ----------
GEMINI_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_KEY:
    print('❌ GEMINI_API_KEY חסר. קבע ב-PowerShell:  setx GEMINI_API_KEY "YOUR_KEY"  ואז פתח מסוף מחדש')
    sys.exit(1)
genai.configure(api_key=GEMINI_KEY)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")


# ---------- Google Login (Desktop) ----------
def google_login() -> Tuple[str, str]:
    """
    פותח דפדפן להתחברות Google, מחזיר (name, email).
    דורש client_secret.json של OAuth Desktop (Credentials → OAuth client ID → Desktop app).
    """
    if not os.path.exists(CLIENT_SECRETS_FILE):
        print(f"❌ לא נמצא קובץ OAuth: {CLIENT_SECRETS_FILE}")
        print("ב-GCP Console צור OAuth Client מסוג Desktop והורד לכאן כ-client_secret.json, או עדכן CLIENT_SECRETS_FILE לנתיב מלא.")
        sys.exit(1)

    flow = InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE,
        scopes=[
            "openid",
            "https://www.googleapis.com/auth/userinfo.email",
            "https://www.googleapis.com/auth/userinfo.profile",
        ],
    )
    creds = flow.run_local_server(port=0, prompt="consent")

    # userinfo
    resp = requests.get(
        "https://www.googleapis.com/oauth2/v3/userinfo",
        headers={"Authorization": f"Bearer {creds.token}"}
    )
    resp.raise_for_status()
    info = resp.json()
    email = info["email"]
    name  = info.get("name") or email.split("@")[0]

    if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
        raise ValueError("אימייל לא תקין מה-Google userinfo")
    return name, email


# ---------- Firestore helpers ----------
def _qa_col(user_email: str):
    return db.collection("users").document(user_email).collection("qa")

@_fs.transactional
def _next_idx_txn(txn, user_email: str) -> int:
    counter_ref = db.collection("counters").document(user_email)
    snap = counter_ref.get(transaction=txn)
    last = snap.to_dict().get("last", 0) if snap.exists else 0
    idx = last + 1
    txn.set(counter_ref, {"last": idx})
    return idx

def get_next_index(user_email: str) -> int:
    return _next_idx_txn(db.transaction(), user_email)

def save_question(user_email: str, user_name: str, question: str, answer: str, idx: int):
    _qa_col(user_email).document(str(idx)).set({
        "email":   user_email,
        "name":    user_name,
        "question": question,
        "answer":   answer,
        "title":    None,                 # Cloud Run Trigger ימלא לאחר create
        "created":  datetime.now(timezone.utc),
    })

def list_qas(user_email: str):
    docs = _qa_col(user_email).order_by("created").stream()
    any_row = False
    for doc in docs:
        d = doc.to_dict()
        # כותרת אם קיימת; אחרת – שורת הפתיח של השאלה
        t = (d.get("title") or d.get("question") or "").strip().splitlines()[0]
        t = t.replace("\u200f", "").replace("\u200e", "")  # ניקוי תווי כיווניות אם נכנסו
        if len(t) > 90:
            t = t[:90] + "…"
        print(f"[{doc.id}] {t}")
        any_row = True
    if not any_row:
        print("אין רשומות עדיין.")

def retrieve_qa(user_email: str, idx_str: str):
    snap = _qa_col(user_email).document(idx_str).get()
    if not snap.exists:
        print("⚠️ אינדקס לא נמצא.")
        return
    d = snap.to_dict()
    print("\n— שאלה —")
    print(d["question"])
    print("\n— תשובה —")
    print(d["answer"])
    print("")

# (אופציונלי) לאיפוס מלא לפני הדגמה – זה ימחק את כל הנתונים שלך ויאפס את המונה
def reset_user_data(user_email: str):
    db.collection("counters").document(user_email).delete()
    for doc in _qa_col(user_email).stream():
        doc.reference.delete()
    print("✅ נמחקו כל הנתונים של המשתמש. האינדקס הבא יהיה 1.")


# ---------- Main CLI ----------
def main():
    try:
        print("🌐 פותח דפדפן ל-Google Login...")
        name, email = google_login()
    except Exception as e:
        print("❌ Google Login נכשל:", e)
        sys.exit(1)

    print(f"✅ מחובר כ: {name} <{email}>")
    print("פקודות: list / retrieve <index> / exit\nהקלדה חופשית = שאלה ל-AI.")

    while True:
        try:
            user_in = input("שאלה: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nלהתראות!")
            break

        if not user_in:
            continue

        cmd = user_in.lower()
        if cmd == "exit":
            break
        elif cmd == "list":
            list_qas(email)
        elif cmd.startswith("retrieve"):
            parts = user_in.split()
            if len(parts) != 2 or not parts[1].isdigit():
                print("שימוש: retrieve <index>  (לדוגמה: retrieve 3)")
                continue
            retrieve_qa(email, parts[1])
        # (אופציונלי) איפוס לפני הדגמה: הקלד reset_user
        elif cmd == "reset_user":
            reset_user_data(email)
        else:
            # שאלה ל-Gemini
            try:
                answer = gemini_model.generate_content(user_in).text
            except Exception as e:
                print("❌ שגיאה בפנייה ל-Gemini:", e)
                continue

            print("\n— תשובת AI —")
            print(answer)
            print("")

            # שמירה ל-Firestore עם אינדקס אטומי למשתמש
            try:
                idx = get_next_index(email)
                save_question(email, name, user_in, answer, idx)
                print(f"✅ נשמר כ-Index {idx}\n")
            except Exception as e:
                print("❌ שגיאה בשמירה ל-Firestore:", e)

if __name__ == "__main__":
    main()


🌐 פותח דפדפן ל-Google Login...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=713266721170-gp62shs3kvngmhta78m0i1ji5hug1the.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65092%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile&state=zeSzcuxx8YoKXO4GRLS7aacMJBdg1N&prompt=consent&access_type=offline
✅ מחובר כ: גיא פרבשטיין <guymkc7@gmail.com>
פקודות: list / retrieve <index> / exit
הקלדה חופשית = שאלה ל-AI.


שאלה:  מה זה מחשוב ענן בקיצור?



— תשובת AI —
מחשוב ענן הוא שימוש ברשתות מחשבים מרוחקות (בדרך כלל האינטרנט) לאחסון, גישה וניתוח נתונים, במקום להשתמש במחשבים מקומיים או שרתים פיזיים.  בפשטות, זה כמו לשכור כוח עיבוד ואחסון במקום לרכוש אותו.


✅ נשמר כ-Index 14



שאלה:  מה זה בינה מלאכותית בקיצור?



— תשובת AI —
בינה מלאכותית (AI) היא תחום במדעי המחשב שמטרתו לפתח מערכות מחשב שיכולות לבצע משימות הדורשות אינטליגנציה אנושית, כמו למידה, תכנון ופתרון בעיות.


✅ נשמר כ-Index 15



...

[1] מה זה מחשוב ענן בקיצור?
[2] מה זה בינה מלאכותית?
[3] מה זה מחשוב ענן בקיצור
[4] מה זה בינה מלאכותית
[5] מי היה איינשטיין?
[6] מה זה מחשוב ענן?
[7] מה זה בינה מלאכותית?
[8] מי היה אלברט אינשטיין?
[9] מה זה מחשוב ענן בקיצור?
[10] מה זה בינה מלאכותית בקיצור?
[11] מי היה אלברט אינשטיין?
[12] מה זה מחשוב ענן בקיצור?
[13] מה זה בינה מלאכותית בקיצור?
[14] מה זה מחשוב ענן בקיצור?
[15] מה זה בינה מלאכותית בקיצור?
[16] מי היה אלברט אינשטיין


שאלה:  retrieve 15



— שאלה —
מה זה בינה מלאכותית בקיצור?

— תשובה —
בינה מלאכותית (AI) היא תחום במדעי המחשב שמטרתו לפתח מערכות מחשב שיכולות לבצע משימות הדורשות אינטליגנציה אנושית, כמו למידה, תכנון ופתרון בעיות.




שאלה:  retrieve 20


⚠️ אינדקס לא נמצא.
